In [1]:
import pandas as pd
# Read dataframe
malConll = pd.read_csv('malExpanded.conllu')
malConll['word'] = malConll['word'].astype(str)
malConll['lemma'] = malConll['lemma'].astype(str)
malConll['DEPS'] = malConll['DEPS'].astype(str)
#Convert all deps to lowecase
malConll['DEPS'] = malConll['DEPS'].str.lower()


malConll.loc[malConll['word']=='ഉള്ള','lemma'] = 'ഉള്'

repDict = {'k1' : ['k1*u','k1a','k1s','k1u','K1U','K1u','ras-k1'], 
           'k2':['k2g','k2p','k2s','k2u','K2U','K2u'], 
           'k4':['k4a','k4u','rask4'],
          'k7':['k7a','k7p','k7t'],
          
           'vmod':['vmod-emph', 'vmod_emph', 'vmodemph', 'vmod-relc', 'vmod-emp', 'vmod_relc',
                   'vmod-enmph', 'vmode-emph','vnmod-emph','vmood-emph'],
        'nmod':[ 'nmod-emph', 'vnmod-emph', 'nmod__k1inv', 'nmod-relc', 'nmod-emp','nmod__relc']
          }

for item in repDict:
    for stuff in repDict[item]:
        malConll.loc[malConll['DEPS']==stuff,'DEPS'] = item
malConll.groupby('DEPS').count().sort_values('sentID',ascending=False)

/home/ashim/indicmt/env_mt/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,sentID,wordPosition,word,lemma,UPOS,XPOS,FEATS,HEAD,DEPREL,MISC,category,gender,number,person,case,vib,tam
DEPS,,,,,,,,,,,,,,,,,,
k1,26465,26465,26465,26465,26465,26462,26465,26465,26465,26465,26465,26031,19071,18842,18797,18791,18463,18774
root,14063,14063,14063,14063,14063,14058,14063,14063,14063,14063,14063,13332,60,27,26,25,6422,6657
k7,13998,13998,13998,13998,13998,13997,13998,13998,13998,13998,13998,13855,9757,9724,9720,9707,9591,9789
rsym,13277,13277,13277,13277,13277,13271,13277,13277,13277,13277,13277,13193,40,18,18,18,29,29
k2,11249,11249,11249,11249,11249,11245,11249,11249,11249,11249,11249,11123,8023,7976,7967,7964,7865,7992
nmod,11245,11245,11245,11245,11245,11244,11245,11245,11245,11245,11245,11069,2659,2615,2615,2612,6851,10117
vmod,10441,10441,10441,10441,10441,10439,10441,10441,10441,10441,10441,10168,289,235,232,233,6448,9321
r6,8925,8925,8925,8925,8925,8925,8925,8925,8925,8925,8925,8851,7330,7324,7310,7309,7402,7436
k4,5457,5457,5457,5457,5457,5456,5457,5457,5457,5457,5457,5399,4026,4016,4012,4011,4082,4090


In [2]:
# malConll[malConll['word'].str.len() <= 1]['word'].unique()
wg = malConll.groupby('DEPS').count().sort_values('word')
wgi = (wg[wg['lemma'] < 400]['lemma'])
sent_ids_remove = malConll[malConll['DEPS'].isin(wgi.index)]['sentID'].unique()
df = malConll[~malConll['sentID'].isin(sent_ids_remove)]
df.groupby('DEPS').count().sort_values('sentID',ascending=False)

,Unnamed: 0,sentID,wordPosition,word,lemma,UPOS,XPOS,FEATS,HEAD,DEPREL,MISC,category,gender,number,person,case,vib,tam
DEPS,,,,,,,,,,,,,,,,,,
k1,25937,25937,25937,25937,25937,25934,25937,25937,25937,25937,25937,25508,18674,18447,18402,18399,18071,18379
root,13773,13773,13773,13773,13773,13768,13773,13773,13773,13773,13773,13058,59,26,25,24,6288,6517
k7,13728,13728,13728,13728,13728,13727,13728,13728,13728,13728,13728,13589,9578,9545,9542,9528,9417,9594
rsym,13018,13018,13018,13018,13018,13012,13018,13018,13018,13018,13018,12937,37,18,18,18,29,29
k2,11026,11026,11026,11026,11026,11022,11026,11026,11026,11026,11026,10905,7854,7810,7801,7798,7705,7826
nmod,10972,10972,10972,10972,10972,10971,10972,10972,10972,10972,10972,10801,2602,2562,2562,2559,6684,9869
vmod,10232,10232,10232,10232,10232,10230,10232,10232,10232,10232,10232,9964,276,224,221,222,6312,9133
r6,8747,8747,8747,8747,8747,8747,8747,8747,8747,8747,8747,8673,7175,7169,7158,7157,7245,7279
k4,5380,5380,5380,5380,5380,5379,5380,5380,5380,5380,5380,5322,3962,3952,3948,3947,4018,4026


In [4]:
# Remove Punctuation words
regex_punc_str = '[\x00-\x2F\x3A-\x40\x5B-\x60-\x7B-\x7F]'
df['word'] = df['word'].str.replace(regex_punc_str, '', regex=True)
df['lemma'] = df['lemma'].str.replace(regex_punc_str, '', regex=True)
punc_remove_index = df[(df['word'] == '') | (df['lemma'] == '')].index
df_new = df[~(df.index.isin(punc_remove_index))]
remove_upos = ['RD__PUNC', 'RD_SYM', 'RD_PUNC','RD__SYM']
remove_upos_index = df_new[df_new['UPOS'].isin(remove_upos)].index
df_new = df_new[~(df_new.index.isin(remove_upos_index))]


# Remove sentences with nan at word or lemma
word_lemma_nan_sent_ids = df_new[(df_new['lemma'].str.lower() == 'nan') | (df_new['word'].str.lower() == 'nan')]['sentID'].unique()
df_new = df_new[~(df_new['sentID'].isin(word_lemma_nan_sent_ids))]
df_new['sentID'].unique().shape

#remove deprel undef
sent_ids_remove = df_new[df_new['DEPS']=='undef']['sentID'].unique()
df_new = df_new[~df_new['sentID'].isin(sent_ids_remove)]

df_new.groupby('DEPS').count().sort_values('sentID',ascending=False), df_new['sentID'].unique().shape
# df_new[(df_new['word'] == '') | (df_new['lemma'] == '')]['sentID'].unique().shape == (0,)
# df_new[df_new['word'].str.len() <= 1]['word'].unique()

/home/ashim/indicmt/env_mt/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/ashim/indicmt/env_mt/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


(      Unnamed: 0  sentID  wordPosition   word  lemma   UPOS   XPOS  FEATS  \
 DEPS                                                                        
 k1         22137   22137         22137  22137  22137  22137  22137  22137   
 root       12111   12111         12111  12111  12111  12111  12111  12111   
 k7         11789   11789         11789  11789  11789  11789  11789  11789   
 k2          9661    9661          9661   9661   9661   9661   9661   9661   
 nmod        9042    9042          9042   9042   9042   9042   9042   9042   
 vmod        8964    8964          8964   8964   8964   8963   8964   8964   
 r6          7242    7242          7242   7242   7242   7242   7242   7242   
 k4          4812    4812          4812   4812   4812   4812   4812   4812   
 pof         2976    2976          2976   2976   2976   2976   2976   2976   
 ccof        2234    2234          2234   2234   2234   2234   2234   2234   
 k5          2001    2001          2001   2001   2001   2001   2

In [ ]:
# Save 10 examples of each dependency type 

# deps = ['k1', 'k5', 'k7', 'k7p', 'k4', 'k1s', 'k2', 'k7t', 'k2s', 'k1u', 'k3', 'k2u', 'K1U', 'k2p', 'k4a', 'K1u', 'k1*u', 'kiu', 'k7a', 'k4u', 'K2U', 'k2g', 'K2u', 'k6',  'k1a']
# group_deps = df_new.groupby('DEPS')
# deps_sentes = {}
# for dep in deps:
#     deps_sentes[dep] = []
# for dep in deps:
#     count = 10
#     gp_index = group_deps.groups[dep]
#     sent_ids = list(df_new[df_new.index.isin(gp_index)]['sentID'].unique())
#     if len(sent_ids) < 10:
#         deps_sentes[dep] = sent_ids
#     else:
#         deps_sentes[dep] = sent_ids[:10]
# s = ",".join(["None" for i in range(df_new.shape[1]-1)])
# with open('dependency_examples.csv', 'w') as fp:
#     fp.write("None,"+ s + "\n")
# with open('dependency_examples.csv', 'a') as fp:
#     for dep in deps:
#         fp.write(str(dep) + "," + s + "\n")
#         df_new.to_csv(fp, header=False)
#     fp.write("\n\n")

In [61]:
df_new[df_new['word'].str.len() <= 1]['word'].unique()
#added one more unecessary case of long word 118445 and 7489
sentToremove = df_new[df_new['Unnamed: 0'].isin([7489,118445])]['sentID'].unique()
df_new = df_new[~(df_new['sentID'].isin(sentToremove))]
# df_new['sentID'].unique().shape
df_new.groupby('vib').count().sort_values('sentID',ascending=False)

,Unnamed: 0,sentID,wordPosition,word,lemma,UPOS,XPOS,FEATS,HEAD,DEPREL,DEPS,MISC,category,gender,number,person,case,tam,source,target
vib,,,,,,,,,,,,,,,,,,,,
0,25308,25308,25308,25308,25308,25308,25308,25308,25308,25308,25308,25308,25300,25269,25270,25273,25266,25232,25308,25308
ഇല്‍,4858,4858,4858,4858,4858,4858,4858,4858,4858,4858,4858,4858,4858,4829,4824,4828,4814,4858,4858,4858
കള്‍,3383,3383,3383,3383,3383,3383,3383,3383,3383,3383,3383,3383,3383,3364,3356,3364,3364,3375,3383,3383
ഉന്ന,2728,2728,2728,2728,2728,2728,2728,2728,2728,2728,2728,2728,2726,2,2,2,2,2726,2728,2728
ഇ,2634,2634,2634,2634,2634,2634,2634,2634,2634,2634,2634,2634,2630,3,4,4,4,2624,2634,2634
ടെ,2399,2399,2399,2399,2399,2399,2399,2399,2399,2399,2399,2399,2396,2369,2368,2365,2368,2395,2399,2399
ഉ്,1820,1820,1820,1820,1820,1820,1820,1820,1820,1820,1820,1820,1820,1715,1711,1715,1715,1820,1820,1820
ഉന്നു,1231,1231,1231,1231,1231,1231,1231,1231,1231,1231,1231,1231,1228,2,2,2,2,1228,1231,1231
ഉടെ,1054,1054,1054,1054,1054,1054,1054,1054,1054,1054,1054,1054,1054,1032,1032,1032,1032,1054,1054,1054


In [62]:
import importlib
import wordpairalignment #import the module here, so that it can be reloaded.
importlib.reload(wordpairalignment)

<module 'wordpairalignment' from '/home/ashim/indicmt/wordpairalignment.py'>

In [13]:
df = df_new
# df = df.head(50000)
df['source'] = df['lemma']
df['target'] = df['word']
df_align= df.apply(wordpairalignment.iterLCS, axis=1)
df_align['extracted'] = df_align['aligned'].str.extract('INS\((.+?)\)')
df_align.shape

SW1 ഇന്രെര്ഫെറോന്
SW2 ഇന്രെര്ഫെറോന്
common ഇന്രെര്ഫെറോന്
ordermatchilla ['1', '൯'] ['1']
ordermatchilla ['1', '൯'] ['1']
ordermatchilla ['1', '൯'] ['1']
ordermatchilla ['1', '൯'] ['1']
ordermatchilla ['1', '൯'] ['1']
ordermatchilla ['2'] []
ordermatchilla ['1', '2', '൯'] ['1', '2']
ordermatchilla ['1', '൯'] ['1']
ordermatchilla ['1', '൯'] ['1']
ordermatchilla ['1', '൯'] ['1']
ordermatchilla ['1', '൯'] ['1']
ordermatchilla ['1', '൯'] ['1']
ordermatchilla ['1', '൯'] ['1']
ordermatchilla ['1', '2', '3', '4'] ['1', '4', '3', '2']
SW2 നേര്ത്ത്നേര്ത്ത്
common നേര്ത്ത്നേര്ത്ത്
ordermatchilla ['2', '3', '1'] ['2', '1', '3']
SW2 താത്പര്യ
common താത്പര്യ
ordermatchilla ['2', '1', '3'] ['2', '3', '1']
SW2 ആസ്രരമെഡിസിര്രി
common ആസ്രരമെഡിസിര്രി
here സര മത്സര മല്‍സരത്തില്‍
ordermatchilla ['1'] ['1', '27']
ordermatchilla ['57'] []
here ര് നിഷ്ക്കര്ഷി നിഷ്‌കര്‍ഷിക്കുന്നത്
here ഷി നിഷ്ക്കര്ഷി നിഷ്‌കര്‍ഷിക്കുന്നത്
ordermatchilla ['1', '2'] ['1', '47', '2']
ordermatchilla ['1', '2', '3'] ['1', '3', '2']

(98420, 28)

In [14]:
df_align[['Unnamed: 0', 'wordPosition', 'sentID', 'word', 'lemma', 'UPOS', 'XPOS', 'DEPS', 'vib', 'extracted']]

,Unnamed: 0,wordPosition,sentID,word,lemma,UPOS,XPOS,DEPS,vib,extracted
0,0,1,malayalam/ilci_tourism_pos_chunk_validated/Dep...,ഗഡ്ഡ്മുക്തേശ്വറിന്‍റെ,ഗഡ്ഡ്മുക്തേശ്വറ്,N__NN,_,r6,ടെ,ിന്‍റെ
1,1,2,malayalam/ilci_tourism_pos_chunk_validated/Dep...,വര്‍ണ്ണന,വര്‍ണ്ണന,N_NN,_,k1,0,NaN
2,2,3,malayalam/ilci_tourism_pos_chunk_validated/Dep...,ശിവപുരാണത്തില്‍‍,ശിവപുരാണ,N__NN,_,k5,ഇല്‍,ത്തില്‍‍
3,3,3,malayalam/ilci_tourism_pos_chunk_validated/Dep...,നിന്ന്,നിന്ന്,PSP,_,k5,NaN,NaN
4,4,4,malayalam/ilci_tourism_pos_chunk_validated/Dep...,ലഭിക്കുന്നത്,ലഭി,V__VM__VNF,_,vmod,ഉന്ന,ക്കുന്നത്
5,5,5,malayalam/ilci_tourism_pos_chunk_validated/Dep...,ആണ്,ആണ്,V__VM__VF,_,root,NaN,NaN
7,7,1,malayalam/ilci_tourism_pos_chunk_validated/Dep...,ചതുര്‍ദശിയുടെ,ചതുര്‍ദശി,N__NN,_,r6,ടെ,യുടെ
8,8,2,malayalam/ilci_tourism_pos_chunk_validated/Dep...,സന്ധ്യയില്‍,സന്ധ്യ,N__NN,_,k7,ഇല്‍,യില്‍
9,9,3,malayalam/ilci_tourism_pos_chunk_validated/Dep...,യാഗാന്ത്യത്തില്‍,യാഗാന്ത്യ,N__NN,_,k7,ഇല്‍,ത്തില്‍
10,10,4,malayalam/ilci_tourism_pos_chunk_validated/Dep...,ആ,ആ,DM,_,k4,NaN,ആ


In [66]:
# df_align.groupby('vib').count().sort_values('sentID',ascending=False)
df_align[(df_align['vib'] == '0' )| (df_align['vib'] == 'nan')].shape

(59777, 28)

In [90]:
df_align['extracted'] = df_align['extracted'].astype(str)
df_align['vib'] = df_align['vib'].astype(str)
df_align.loc[df_align['vib'].str.lower() == 'nan', 'vib'] = df_align['extracted']
df_align['extracted'] = df_align['aligned'].str.extract('INS\((.+?)\)')
df_align.loc[df_align['vib'].str.lower() == 'nan', 'vib'] = df_align['extracted']
df_align.loc[df_align['vib'].str.lower() == '0', 'vib'] = df_align['extracted']
columns = ['Unnamed: 0', 'wordPosition', 'sentID', 'word', 'lemma', 'UPOS', 'XPOS', 'DEPS', 'vib', 'extracted', 'aligned']
df_align.loc[df_align['word'] == df_align['lemma'], 'vib'] = 'SAME'
df_align[(df_align['vib'] == '0' )| (df_align['vib'] == 'nan') | (df_align['vib'].isnull())].shape
# df_align[(df['word'] == df['lemma'])].groupby('vib').count().sort_values('sentID',ascending=False)
# df_align[df_align['vib'].isnull()][columns]

,Unnamed: 0,wordPosition,sentID,word,lemma,UPOS,XPOS,DEPS,vib,extracted,aligned
4799,4799,2,malayalam/full data as on 25-01-2017/not valid...,എത്രയോ,എത്രയോടെ,PR_PRQ,_,nmod,NaN,NaN,എത്രയോDEL(ടെ)
5550,5550,3,malayalam/full data as on 25-01-2017/not valid...,പോളിപ്,പോളിപ്പ്,N_NN,_,rs,NaN,NaN,പോളിപ്DEL(പ്)
5943,5943,6,malayalam/full data as on 25-01-2017/not valid...,വിറ്വരവ്,വിറ്റുവരവ്,N__NN,_,k1,NaN,NaN,വിറ്DEL(റു)വരവ്
6692,6692,6,malayalam/full data as on 25-01-2017/not valid...,2000,2000കോടിഡോളര്,QT_QTC,_,k1,NaN,NaN,2000DEL(കോടിഡോളര്)
7138,7138,4,malayalam/full data as on 25-01-2017/not valid...,കുത്തിവയ്പ്,കുത്തിവയ്പ്പ്,N_NN,_,k2,NaN,NaN,കുത്തിവയ്പ്DEL(പ്)
7432,7432,4,malayalam/full data as on 25-01-2017/not valid...,വെച്ച,വെച്ച്,V_VM_VNF,_,nmod,NaN,NaN,വെച്ചDEL(്)
8321,8321,1,malayalam/full data as on 25-01-2017/not valid...,രണ്ടു,രണ്ടു്,QT_QTC,_,k4,NaN,NaN,രണ്ടുDEL(്)
8498,8498,5,malayalam/full data as on 25-01-2017/not valid...,നിയന്ത്രാതീതം,നിയന്ത്രണാതീതം,N_NN,_,pof,NaN,NaN,നിയന്ത്രDEL(ണ)ാതീതം
8509,8509,1,malayalam/full data as on 25-01-2017/not valid...,ഗ്രീക്,ഗ്രീക്ക്,N__NNP,_,nmod,NaN,NaN,ഗ്രീക്DEL(ക്)
9506,9506,1,malayalam/full data as on 25-01-2017/not valid...,രണ്ടാം,രണ്ടാംലോക,QT__QTO,_,r6,NaN,NaN,രണ്ടാംDEL(ലോക)


In [93]:
df_align[columns]

,Unnamed: 0,wordPosition,sentID,word,lemma,UPOS,XPOS,DEPS,vib,extracted,aligned
0,0,1,malayalam/ilci_tourism_pos_chunk_validated/Dep...,ഗഡ്ഡ്മുക്തേശ്വറിന്‍റെ,ഗഡ്ഡ്മുക്തേശ്വറ്,N__NN,_,r6,ടെ,NaN,ഗഡ്ഡ്മുക്തേശ്വറDEL(്)INS(ിന്‍റെ)
1,1,2,malayalam/ilci_tourism_pos_chunk_validated/Dep...,വര്‍ണ്ണന,വര്‍ണ്ണന,N_NN,_,k1,SAME,NaN,വര്‍ണ്ണന
2,2,3,malayalam/ilci_tourism_pos_chunk_validated/Dep...,ശിവപുരാണത്തില്‍‍,ശിവപുരാണ,N__NN,_,k5,ഇല്‍,NaN,ശിവപുരാണINS(ത്തില്‍‍)
3,3,3,malayalam/ilci_tourism_pos_chunk_validated/Dep...,നിന്ന്,നിന്ന്,PSP,_,k5,SAME,NaN,നിന്ന്
4,4,4,malayalam/ilci_tourism_pos_chunk_validated/Dep...,ലഭിക്കുന്നത്,ലഭി,V__VM__VNF,_,vmod,ഉന്ന,NaN,ലഭിINS(ക്കുന്നത്)
5,5,5,malayalam/ilci_tourism_pos_chunk_validated/Dep...,ആണ്,ആണ്,V__VM__VF,_,root,SAME,NaN,ആണ്
7,7,1,malayalam/ilci_tourism_pos_chunk_validated/Dep...,ചതുര്‍ദശിയുടെ,ചതുര്‍ദശി,N__NN,_,r6,ടെ,NaN,ചതുര്‍ദശിINS(യുടെ)
8,8,2,malayalam/ilci_tourism_pos_chunk_validated/Dep...,സന്ധ്യയില്‍,സന്ധ്യ,N__NN,_,k7,ഇല്‍,NaN,സന്ധ്യINS(യില്‍)
9,9,3,malayalam/ilci_tourism_pos_chunk_validated/Dep...,യാഗാന്ത്യത്തില്‍,യാഗാന്ത്യ,N__NN,_,k7,ഇല്‍,NaN,യാഗാന്ത്യINS(ത്തില്‍)
10,10,4,malayalam/ilci_tourism_pos_chunk_validated/Dep...,ആ,ആ,DM,_,k4,SAME,ആ,DEL(ആ)INS(ആ)


In [98]:
df = df_align
deps_to_include = ['k1', 'k2', 'k3', 'k4', 'k5', 'r6', 'k7', 'vmod', 'nmod']
df[df['DEPS'].isin(deps_to_include)]['sentID'].unique().shape, df['sentID'].unique().shape

((11629,), (11651,))

In [129]:
import numpy as np
cl = ['Unnamed: 0', 'sentID', 'word', 'lemma', 'DEPS', 'vib']
lemma_dep_group_by = df[cl].groupby(['lemma', 'DEPS'])
lemma_group_by = df[cl].groupby('lemma')
sentToremove = df[df['vib'].isnull()]['sentID'].unique()
df = df[~(df['sentID'].isin(sentToremove))]

# lemma_dep_group_by.groups.keys()
lemma_dep_group_by.get_group(('അതിപുരാതന', 'k1'))['vib'].reset_index()
# lemma_dep_group_by['vib'].apply(np.unique).reset_index()
# df[df['vib'].isnull()].shape
# lemma_group_by.count().sort_values('sentID',ascending=False)

,index,vib
0,39760,SAME
1,81304,SAME
2,115489,SAME


In [240]:
df1 = df[['lemma', 'DEPS', 'vib']]
df2 = df[['lemma', 'vib']]

new_right = pd.merge(df1, df2, how='outer', on =['lemma'])
new_right

,lemma,DEPS,vib_x,vib_y
0,ഗഡ്ഡ്മുക്തേശ്വറ്,r6,ടെ,ടെ
1,ഗഡ്ഡ്മുക്തേശ്വറ്,r6,ടെ,ഇല്‍
2,ഗഡ്ഡ്മുക്തേശ്വറ്,r6,ടെ,ഇല്‍
3,ഗഡ്ഡ്മുക്തേശ്വറ്,r6,ടെ,ഇലെ
4,ഗഡ്ഡ്മുക്തേശ്വറ്,r6,ടെ,ഇലെ
5,ഗഡ്ഡ്മുക്തേശ്വറ്,r6,ടെ,ഇലെ
6,ഗഡ്ഡ്മുക്തേശ്വറ്,r6,ടെ,ഇലെ
7,ഗഡ്ഡ്മുക്തേശ്വറ്,r6,ടെ,ടെ
8,ഗഡ്ഡ്മുക്തേശ്വറ്,r6,ടെ,ഇല്‍
9,ഗഡ്ഡ്മുക്തേശ്വറ്,r6,ടെ,ഇല്‍


In [243]:
df_final = pd.merge(new_right, df1, right_on=['lemma', 'DEPS', 'vib'], left_on=['lemma', 'DEPS', 'vib_y'], how="outer", indicator=True
              ).query('_merge=="left_only"')['lemma', 'DEPS', 'vib_x', 'vib_y']
df_final.shape

MemoryError: 

In [242]:
# Following is an alternate way, also Memory Error

# df_common = pd.merge(new_right, df1, right_on=['lemma', 'DEPS', 'vib'], left_on=['lemma', 'DEPS', 'vib_y'], how='inner')[['lemma', 'DEPS', 'vib_y', 'vib_x']]
# df_common.columns = ['lemma', 'DEPS', 'vib1', 'vib2']
# new_right.columns = ['lemma', 'DEPS', 'vib1', 'vib2']

# # Now remove common rows
# df_remain = df_right.merge(df_inter.drop_duplicates(), on=['lemma', 'DEPS', 'vib1', 'vib2'], 
#                    how='left', indicator=True).query('_merge=="left_only"')
# df_remain

MemoryError: 